In [1]:
import os
import csv
import cv2
import re
import numpy as np
import matplotlib.pyplot as plt
from random import shuffle
import sklearn

%matplotlib inline

samples = []

def read_dataset(directory):
    with open(directory+'driving_log.csv') as csvfile:
        reader = csv.reader(csvfile)
        for line in reader: 
            for angle in range(3):
                path = line[angle]
                filename = path.split('/')[-1]
                filename = filename.split('\\')[-1]
                path = directory+'IMG/'+filename
                line[angle] = path
            samples.append(line)


In [2]:
def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]
            
            images = []
            steering_angles = []
            throttles = []
            labels = []
            
            
            for batch_sample in batch_samples:
                if(batch_sample[3] == 'steering'):
                    continue
                
                steering = float(batch_sample[3])
                throttle = float(batch_sample[4])
                
                correction = 0.1 # this is a parameter to tune
                
                for angle in range(3):
                    
                    path = batch_sample[angle]

                    image = cv2.imread(path)
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

                    images.append(image)
                    throttles.append(throttle)

                # create adjusted steering measurements for the side camera images
                    if(angle == 1):
                        steering += correction
                    elif (angle == 2):
                        steering -= correction

                    steering_angles.append(steering)
                    
                    labels.append((steering, throttle))

                    image_flipped = image.copy()
                    image_flipped = cv2.flip(image,1)
                    steering_flipped = -steering
                    images.append(image_flipped)
                    steering_angles.append(steering_flipped)
                    labels.append((steering, throttle))

            # trim image to only see section with road
            X_train = np.array(images)
#             y_train = np.array(steering_angles)  
            y_train = np.array(labels)
#             print ('y_train = ',y_train)
#             print('X_train: ',X_train.shape)
#             print('y_train: ',(y_train.shape))
            
            yield sklearn.utils.shuffle(X_train, y_train)

In [3]:
# X_train = np.array(images)
# y_train = np.array(measurements)

read_dataset('data_custom_3/')
shuffle(samples)
from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=32)
validation_generator = generator(validation_samples, batch_size=32)

# print (len(train_generator), len(validation_generator))
# print(train_generator.shape, validation_generator.shape)

In [4]:
from keras.models import Model, Sequential
from keras.layers import Input, Flatten, Dense, Activation, Lambda, Dropout, Cropping2D
from keras.layers.convolutional import Convolution2D
from keras.layers.pooling import MaxPooling2D


model = Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((60,20),(0,0))))
model.add(Convolution2D(24,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(36,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(48,5,5,subsample=(2,2),activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Convolution2D(64,3,3,activation='relu'))
model.add(Flatten())
model.add(Dense(100))
model.add(Dense(50))
model.add(Dense(10))
model.add(Dense(output_dim = 2))

model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, samples_per_epoch = len(train_samples), validation_data=validation_generator, nb_val_samples=len(validation_samples), nb_epoch=3)

model.save('model.h5')


Using TensorFlow backend.


Epoch 1/3
21312/21338 [============================>.] - ETA: 0s - loss: 0.1161

/home/carnd/anaconda3/envs/carnd-term1/lib/python3.5/site-packages/keras/engine/training.py:1569: UserWarning: Epoch comprised more than `samples_per_epoch` samples, which might affect learning results. Set `samples_per_epoch` correctly to avoid this warning.
  warnings.warn('Epoch comprised more than '


21504/21338 [==============================] - 60s - loss: 0.1157 - val_loss: 0.0732
Epoch 2/3
21504/21338 [==============================] - 57s - loss: 0.0748 - val_loss: 0.0763
Epoch 3/3
21504/21338 [==============================] - 57s - loss: 0.0739 - val_loss: 0.0827
